In [19]:
import argparse

from tqdm import tqdm

In [20]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [22]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load nci


In [23]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [24]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/60 [00:00<?, ?it/s]

epoch:   0 loss:0.699949 auc:0.4809
epoch:  20 loss:0.491143 auc:0.5382
epoch:  40 loss:0.398978 auc:0.5959
epoch:  60 loss:0.355027 auc:0.5986
epoch:  80 loss:0.328544 auc:0.5904
epoch: 100 loss:0.316726 auc:0.6280
epoch: 120 loss:0.305445 auc:0.5953
epoch: 140 loss:0.295142 auc:0.6046
epoch: 160 loss:0.289513 auc:0.5961
epoch: 180 loss:0.285784 auc:0.6021
epoch: 200 loss:0.281481 auc:0.5689
epoch: 220 loss:0.279227 auc:0.5812
epoch: 240 loss:0.276524 auc:0.5996
epoch: 260 loss:0.274319 auc:0.6024
epoch: 280 loss:0.271579 auc:0.5730
epoch: 300 loss:0.271146 auc:0.5603
epoch: 320 loss:0.268952 auc:0.5483
epoch: 340 loss:0.266731 auc:0.5473
epoch: 360 loss:0.266157 auc:0.5409
epoch: 380 loss:0.264434 auc:0.5410
epoch: 400 loss:0.264387 auc:0.5591
epoch: 420 loss:0.261826 auc:0.5582
epoch: 440 loss:0.260071 auc:0.5256
epoch: 460 loss:0.257839 auc:0.5498
epoch: 480 loss:0.257266 auc:0.5452
epoch: 500 loss:0.256213 auc:0.5320
epoch: 520 loss:0.252138 auc:0.5557
epoch: 540 loss:0.252355 auc

  2%|▏         | 1/60 [00:06<06:12,  6.32s/it]

Fit finished.
epoch:   0 loss:0.702829 auc:0.5612
epoch:  20 loss:0.495528 auc:0.5707
epoch:  40 loss:0.404588 auc:0.6298
epoch:  60 loss:0.358548 auc:0.6266
epoch:  80 loss:0.332341 auc:0.5957
epoch: 100 loss:0.313262 auc:0.6233
epoch: 120 loss:0.304157 auc:0.6175
epoch: 140 loss:0.295647 auc:0.6393
epoch: 160 loss:0.291019 auc:0.6520
epoch: 180 loss:0.285583 auc:0.6433
epoch: 200 loss:0.283656 auc:0.6671
epoch: 220 loss:0.279577 auc:0.6374
epoch: 240 loss:0.276467 auc:0.6413
epoch: 260 loss:0.273220 auc:0.6528
epoch: 280 loss:0.272568 auc:0.6704
epoch: 300 loss:0.270162 auc:0.6446
epoch: 320 loss:0.268979 auc:0.6653
epoch: 340 loss:0.266716 auc:0.6593
epoch: 360 loss:0.264714 auc:0.6535
epoch: 380 loss:0.263538 auc:0.6582
epoch: 400 loss:0.261213 auc:0.6785
epoch: 420 loss:0.260793 auc:0.6427
epoch: 440 loss:0.258509 auc:0.6651
epoch: 460 loss:0.259058 auc:0.6718
epoch: 480 loss:0.257222 auc:0.6630
epoch: 500 loss:0.254833 auc:0.6320
epoch: 520 loss:0.253423 auc:0.6113
epoch: 540 los

  3%|▎         | 2/60 [00:13<06:49,  7.05s/it]

...

Fit finished.
epoch:   0 loss:0.703517 auc:0.4030
epoch:  20 loss:0.490532 auc:0.6238
epoch:  40 loss:0.401566 auc:0.6160
epoch:  60 loss:0.356576 auc:0.5940
epoch:  80 loss:0.330184 auc:0.6151
epoch: 100 loss:0.314653 auc:0.6252
epoch: 120 loss:0.303606 auc:0.6226
epoch: 140 loss:0.296480 auc:0.6126
epoch: 160 loss:0.291067 auc:0.6062
epoch: 180 loss:0.289718 auc:0.6361
epoch: 200 loss:0.283308 auc:0.6208
epoch: 220 loss:0.280546 auc:0.6132
epoch: 240 loss:0.278311 auc:0.6248
epoch: 260 loss:0.277041 auc:0.6306
epoch: 280 loss:0.274332 auc:0.6083
epoch: 300 loss:0.272286 auc:0.6146
epoch: 320 loss:0.269412 auc:0.6349
epoch: 340 loss:0.267542 auc:0.6191
epoch: 360 loss:0.265624 auc:0.6199
epoch: 380 loss:0.265019 auc:0.6320
epoch: 400 loss:0.264737 auc:0.6320
epoch: 420 loss:0.262691 auc:0.6352
epoch: 440 loss:0.261792 auc:0.6362
epoch: 460 loss:0.260610 auc:0.6117
epoch: 480 loss:0.256911 auc:0.6111
epoch: 500 loss:0.255407 auc:0.6318
epoch: 520 loss:0.255702 auc:0.5812
epoch: 540 los

 98%|█████████▊| 59/60 [13:59<00:16, 16.21s/it]

Fit finished.
epoch:   0 loss:0.701695 auc:0.5129
epoch:  20 loss:0.494804 auc:0.6559
epoch:  40 loss:0.405447 auc:0.6920
epoch:  60 loss:0.356859 auc:0.6916
epoch:  80 loss:0.330817 auc:0.7087
epoch: 100 loss:0.314865 auc:0.7117
epoch: 120 loss:0.304435 auc:0.7145
epoch: 140 loss:0.297252 auc:0.7138
epoch: 160 loss:0.290489 auc:0.7245
epoch: 180 loss:0.285929 auc:0.7304
epoch: 200 loss:0.285456 auc:0.7253
epoch: 220 loss:0.281727 auc:0.7361
epoch: 240 loss:0.277580 auc:0.7374
epoch: 260 loss:0.276408 auc:0.7372
epoch: 280 loss:0.275014 auc:0.7401
epoch: 300 loss:0.271043 auc:0.7370
epoch: 320 loss:0.270275 auc:0.7412
epoch: 340 loss:0.267450 auc:0.7434
epoch: 360 loss:0.265231 auc:0.7333
epoch: 380 loss:0.263783 auc:0.7423
epoch: 400 loss:0.263498 auc:0.7392
epoch: 420 loss:0.263117 auc:0.7419
epoch: 440 loss:0.263594 auc:0.7356
epoch: 460 loss:0.259311 auc:0.7415
epoch: 480 loss:0.256091 auc:0.7353
epoch: 500 loss:0.255603 auc:0.7353
epoch: 520 loss:0.255049 auc:0.7269
epoch: 540 los

100%|██████████| 60/60 [14:15<00:00, 14.26s/it]

Fit finished.


In [25]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")